<a href="https://colab.research.google.com/github/salmamohsen6/multi-person-pos-estimation-using-movenet-and-MMpose/blob/main/player_pos_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.3 MB 11 kB/s 
     |████████████████████████████████| 394.3 MB 16 kB/s 
     |████████████████████████████████| 2.9 MB 59.5 MB/s 
     |████████████████████████████████| 132 kB 59.2 MB/s 
     |████████████████████████████████| 14.8 MB 57.0 MB/s 
     |████████████████████████████████| 462 kB 75.2 MB/s 
     |████████████████████████████████| 3.8 MB 57.7 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68721 sha256=e51bc541bca4b7866a5f1d6220d2563957bceba627216d648a56e691f1752108
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Atte

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# movenet

In [ ]:
import tensorflow_hub as hub

model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [ ]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [ ]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [ ]:
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [ ]:
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture('/content/1.mp4')


fps = cap.get(cv2.CAP_PROP_FPS)
fourcc =  cv2.VideoWriter_fourcc('P','I','M','1')
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

out = cv2.VideoWriter('/content/movenet1.avi',fourcc,fps,(1080,1920),isColor=True)


for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
    success, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
    #print(frame.shape)

    #frame = cv2.resize(frame, (width, height))
    #print(frame.shape)
    #print((height,width))
    #out.write(frame)

    cv2_imshow( frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# **MMpose**

In [ ]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
%pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
%pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

# install mmdet for inference demo
%pip install mmdet

# clone mmpose repo
%rm -rf mmpose
!git clone https://github.com/open-mmlab/mmpose.git
%cd mmpose

# install mmpose dependencies
%pip install -r requirements.txt

# install mmpose in develop mode
%pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████▌                   | 834.1 MB 1.3 MB/s eta 0:16:11tcmalloc: large alloc 1147494400 bytes == 0x3a278000 @  0x7f6d3c4e5615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |███████████████▉                | 1055.7 MB 1.5 MB/s eta 0:12:20tcmalloc: large alloc 1434370048 bytes == 0x7e8ce000 @  0x7f6d3c4e5615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [ ]:
# Check Pytorch installation
import torch, torchvision

print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose

print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

torch version: 1.10.0+cu111 True
torchvision version: 0.11.0+cu111
mmpose version: 0.27.0
cuda version: 11.1
compiler information: GCC 7.3


In [ ]:
import cv2
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,vis_pose_result, process_mmdet_results)
from mmdet.apis import inference_detector, init_detector
import json as simplejson

local_runtime = False

try:
    from google.colab.patches import cv2_imshow  # for image visualization in colab
except:
    local_runtime = True

pose_config = 'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
det_config = 'demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

# initialize pose model
pose_model = init_pose_model(pose_config, pose_checkpoint)
# initialize detector
det_model = init_detector(det_config, det_checkpoint)

#img = '/content/5f46b5f4132b0210196e5b24_alexander-chung-temporary-lover-basic (1).jpg'

cap = cv2.VideoCapture('/content/2.mp4')

fps = cap.get(cv2.CAP_PROP_FPS)

fourcc =  cv2.VideoWriter_fourcc('P','I','M','1')

out = cv2.VideoWriter('/content/MMpose1.avi',fourcc,fps,(960,540),isColor=True)


for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
# inference detection
  success, frame = cap.read()
  
  # Resize image
  img = frame.copy()
  mmdet_results = inference_detector(det_model, img)

  # extract person (COCO_ID=1) bounding boxes from the detection results
  person_results = process_mmdet_results(mmdet_results, cat_id=1)

  # inference pose
  pose_results, returned_outputs = inference_top_down_pose_model(
      pose_model,
      img,
      person_results,
      bbox_thr=0.3,
      format='xyxy',
      dataset=pose_model.cfg.data.test.type)
  

  np.savetxt('/content/test.txt', pose_results, fmt='%s') 
  
  # show pose estimation results
  vis_result = vis_pose_result(
      pose_model,
      img,
      pose_results,
      dataset=pose_model.cfg.data.test.type,
      show=False)
  
  # reduce image size
  vis_result = cv2.resize(vis_result, dsize=None, fx=0.5, fy=0.5)
  #vis_result = cv2.resize(vis_result, (width, height), interpolation = cv2.INTER_AREA)
  
  #height, width, channels = vis_result.shape
  #print( height, width, channels)


  if local_runtime:
      from IPython.display import Image, display
      import tempfile
      import os.path as osp
      with tempfile.TemporaryDirectory() as tmpdir:
          file_name = osp.join(tmpdir, 'pose_results.png')
          cv2.imwrite(file_name, vis_result)
          display(Image(file_name))
  else:
      #cv2.imwrite('/content/1.jpg', vis_result)
      out.write(vis_result)
      cv2_imshow(vis_result)
      

  if not success:
        break

    

cap.release()
cv2.destroyAllWindows()

In [ ]:
pip install MoviePy
import moviepy.editor as moviepy
clip = moviepy.VideoFileClip("/content/MMpose1.avi")
clip.write_videofile("/content/myvideo.mp4")

[MoviePy] >>>> Building video /content/myvideo.mp4
[MoviePy] Writing video /content/myvideo.mp4


100%|██████████| 67/67 [00:01<00:00, 34.96it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/myvideo.mp4 

